<a href="https://colab.research.google.com/github/Gaurav-phatkare/Retrieval-Augmented-Generation_RAG/blob/main/RAG_Fundamentals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Thu Jul 25 10:32:43 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   61C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [1]:
import torch

## building

1. Open a PDF document
2. Format the text of that PDF suitable for embedding model
3. embed into chunks and then convert it into numerical representation
4. use vector db to search relevent cuunk
5. crete prompt that incoporates the retrived peices of text
6. generate answer

## Import PDF


In [2]:
import os
import requests

pdf_path = "human-nutrition-text.pdf"

# download PDF

if not os.path.exists(pdf_path):
  print("File doesnt exists")

  url = "https://pressbooks.oer.hawaii.edu/humannutrition2/open/download?type=pdf"

  filename = pdf_path
  res = requests.get(url)

  if res.status_code == 200:
    with open(filename, "wb") as file:
      file.write(res.content)
    print(f"file has been downloaded and stored as {filename}")

  else:
    print(f"failed to download file, Status code : {res.status_code}")
else:
  print("file exists")

File doesnt exists
file has been downloaded and stored as human-nutrition-text.pdf


In [3]:
!pip install pyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 81.3 MB/s eta 0:00:00


In [4]:
!pip install tqdm

In [5]:
import fitz
from tqdm.auto import tqdm


def text_formatter(text:str):
  cleaned_text = text.replace("\n", " ").strip()
  return cleaned_text


def open_and_read_pdf(pdf_path):
  doc = fitz.open(pdf_path)

  pages_and_texts = []

  for page_number, page in tqdm(enumerate(doc)):
    text = page.get_text()

    text = text_formatter(text=text)

    pages_and_texts.append({"page_number" : page_number - 41,
                            "page_char_count" : len(text),
                            "page_word_count" : len(text.split(" ")),
                            "page_sentence_count_raw" : len(text.split(". ")),
                            "page_token_count": len(text) / 4,
                            "text" : text})

  return pages_and_texts


pages_and_texts = open_and_read_pdf(pdf_path)
pages_and_texts[:3]




0it [00:00, ?it/s]

[{'page_number': -41,
  'page_char_count': 29,
  'page_word_count': 4,
  'page_sentence_count_raw': 1,
  'page_token_count': 7.25,
  'text': 'Human Nutrition: 2020 Edition'},
 {'page_number': -40,
  'page_char_count': 0,
  'page_word_count': 1,
  'page_sentence_count_raw': 1,
  'page_token_count': 0.0,
  'text': ''},
 {'page_number': -39,
  'page_char_count': 320,
  'page_word_count': 54,
  'page_sentence_count_raw': 1,
  'page_token_count': 80.0,
  'text': 'Human Nutrition: 2020  Edition  UNIVERSITY OF HAWAI‘I AT MĀNOA  FOOD SCIENCE AND HUMAN  NUTRITION PROGRAM  ALAN TITCHENAL, SKYLAR HARA,  NOEMI ARCEO CAACBAY, WILLIAM  MEINKE-LAU, YA-YUN YANG, MARIE  KAINOA FIALKOWSKI REVILLA,  JENNIFER DRAPER, GEMADY  LANGFELDER, CHERYL GIBBY, CHYNA  NICOLE CHUN, AND ALLISON  CALABRESE'}]

In [6]:
import random

random.sample(pages_and_texts, 2)

[{'page_number': 793,
  'page_char_count': 1935,
  'page_word_count': 341,
  'page_sentence_count_raw': 16,
  'page_token_count': 483.75,
  'text': 'provide nutrients, phytochemicals, antioxidants, and the extra 3  mg/day of fiber that is recommended during pregnancy. These  foods also help to build the placenta and supply energy for the  growth of the unborn baby.  During pregnancy, extra protein is needed for the synthesis of  new maternal and fetal tissues. Protein builds muscle and other  tissues, enzymes, antibodies, and hormones in both the mother and  the unborn baby. Additional protein also supports increased blood  volume and the production of amniotic fluid. The RDA of protein  during pregnancy is 71 grams per day, which is 25 grams above the  normal recommendation. Protein should be derived from healthy  sources, such as lean red meat, poultry, legumes, nuts, seeds, eggs,  and fish. Low-fat milk and other dairy products also provide protein,  along with calcium and other nut

In [7]:
import pandas as pd

df = pd.DataFrame(pages_and_texts)
df.head()

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,text
0,-41,29,4,1,7.25,Human Nutrition: 2020 Edition
1,-40,0,1,1,0.00,
2,-39,320,54,1,80.00,Human Nutrition: 2020 Edition UNIVERSITY OF ...
3,-38,212,32,1,53.00,Human Nutrition: 2020 Edition by University of...
4,-37,797,145,2,199.25,Contents Preface University of Hawai‘i at Mā...


In [8]:
df.describe()

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count
count,1208.00000,1208.000000,1208.000000,1208.000000,1208.000000
mean,562.50000,1148.594371,198.889901,9.972682,287.148593
std,348.86387,560.441673,95.747365,6.187226,140.110418
min,-41.00000,0.000000,1.000000,1.000000,0.000000
25%,260.75000,762.750000,134.000000,4.000000,190.687500
50%,562.50000,1232.500000,215.000000,10.000000,308.125000
75%,864.25000,1605.250000,271.250000,14.000000,401.312500
max,1166.00000,2308.000000,429.000000,32.000000,577.000000


In [9]:
# Text processing
from spacy.lang.en import English

nlp = English()


nlp.add_pipe('sentencizer')


doc = nlp("this is a sentence. This is another sentence... . I love tensorflow.")

# assert len(list(doc.sents)) == 3

list(doc.sents)

[this is a sentence., This is another sentence... ., I love tensorflow.]

In [10]:
for item in tqdm(pages_and_texts):
  item['sentences'] =  list(nlp(item['text']).sents)

  item['sentences'] = [str(sentence) for sentence in item['sentences']]

  item['page_sentence_count_spacy'] = len(item['sentences'])



  0%|          | 0/1208 [00:00<?, ?it/s]

In [11]:
random.sample(pages_and_texts, 1)

[{'page_number': 751,
  'page_char_count': 1267,
  'page_word_count': 231,
  'page_sentence_count_raw': 15,
  'page_token_count': 316.75,
  'text': 'with 1 tsp. olive oil,  40  \xa0\xa0\xa0with 1 tsp. sesame seeds  18  ½ c. cooked wild rice  83  \xa0\xa0\xa0with ½ c. chopped kale  18  1 whole-wheat dinner roll  4  \xa0\xa0\xa0with 1 tsp. almond butter  33  691  (Total calories from all meals and  snacks = 1,814)  Discretionary calorie allowance: 186  (Total calories from all meals and snacks = 1,814)  Discretionary calorie allowance: 186  Healthy Eating Index  To assess whether the American diet is conforming to the Dietary  Guidelines, the Center for Nutrition Policy and Promotion (CNPP),  a division of the USDA, uses a standardized tool called the Healthy  Eating Index (HEI)2.  The first HEI was developed in 1995 and revised in 2006. This  tool is a simple scoring system of dietary components. The data for  scoring diets is taken from national surveys of particular population  subgro

In [12]:
df = pd.DataFrame(pages_and_texts)

df.describe()

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,page_sentence_count_spacy
count,1208.00000,1208.000000,1208.000000,1208.000000,1208.000000,1208.000000
mean,562.50000,1148.594371,198.889901,9.972682,287.148593,10.319536
std,348.86387,560.441673,95.747365,6.187226,140.110418,6.300843
min,-41.00000,0.000000,1.000000,1.000000,0.000000,0.000000
25%,260.75000,762.750000,134.000000,4.000000,190.687500,5.000000
50%,562.50000,1232.500000,215.000000,10.000000,308.125000,10.000000
75%,864.25000,1605.250000,271.250000,14.000000,401.312500,15.000000
max,1166.00000,2308.000000,429.000000,32.000000,577.000000,28.000000


In [13]:
df.head()

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,text,sentences,page_sentence_count_spacy
0,-41,29,4,1,7.25,Human Nutrition: 2020 Edition,[Human Nutrition: 2020 Edition],1
1,-40,0,1,1,0.00,,[],0
2,-39,320,54,1,80.00,Human Nutrition: 2020 Edition UNIVERSITY OF ...,[Human Nutrition: 2020 Edition UNIVERSITY OF...,1
3,-38,212,32,1,53.00,Human Nutrition: 2020 Edition by University of...,[Human Nutrition: 2020 Edition by University o...,1
4,-37,797,145,2,199.25,Contents Preface University of Hawai‘i at Mā...,[Contents Preface University of Hawai‘i at M...,2


In [14]:
chunk_size = 10

def split_list(input_list, slice_size = chunk_size):

  return [input_list[i:i+slice_size] for i in range(0, len(input_list), slice_size)]

test_list = list(range(25))

split_list(test_list)

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
 [20, 21, 22, 23, 24]]

In [15]:
for item in tqdm(pages_and_texts):
  item['sentence_chunks'] = split_list(input_list = item['sentences'], slice_size = chunk_size)

  item['num_chunks'] = len(item['sentence_chunks'])

  0%|          | 0/1208 [00:00<?, ?it/s]

In [16]:
random.sample(pages_and_texts, 1)

[{'page_number': 807,
  'page_char_count': 449,
  'page_word_count': 84,
  'page_sentence_count_raw': 3,
  'page_token_count': 112.25,
  'text': 'Image by  Marie  Kainoa  Fialkowski  Revilla / CC  BY 4.0  Infancy  UNIVERSITY OF HAWAI‘I AT MĀNOA FOOD SCIENCE AND HUMAN  NUTRITION PROGRAM AND HUMAN NUTRITION PROGRAM  Diet and nutrition have a major impact on a child’s development  from infancy into the adolescent years. A healthy diet not only  affects growth, but also immunity, intellectual capabilities, and  emotional well-being. One of the most important jobs of parenting  Infancy  |  807',
  'sentences': ['Image by  Marie  Kainoa  Fialkowski  Revilla / CC  BY 4.0  Infancy  UNIVERSITY OF HAWAI‘I AT MĀNOA FOOD SCIENCE AND HUMAN  NUTRITION PROGRAM AND HUMAN NUTRITION PROGRAM  Diet and nutrition have a major impact on a child’s development  from infancy into the adolescent years.',
   'A healthy diet not only  affects growth, but also immunity, intellectual capabilities, and  emotional we

In [17]:
df = pd.DataFrame(pages_and_texts)

df.describe()

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,page_sentence_count_spacy,num_chunks
count,1208.00000,1208.000000,1208.000000,1208.000000,1208.000000,1208.000000,1208.000000
mean,562.50000,1148.594371,198.889901,9.972682,287.148593,10.319536,1.525662
std,348.86387,560.441673,95.747365,6.187226,140.110418,6.300843,0.644397
min,-41.00000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000
25%,260.75000,762.750000,134.000000,4.000000,190.687500,5.000000,1.000000
50%,562.50000,1232.500000,215.000000,10.000000,308.125000,10.000000,1.000000
75%,864.25000,1605.250000,271.250000,14.000000,401.312500,15.000000,2.000000
max,1166.00000,2308.000000,429.000000,32.000000,577.000000,28.000000,3.000000


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1208 entries, 0 to 1207
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   page_number                1208 non-null   int64  
 1   page_char_count            1208 non-null   int64  
 2   page_word_count            1208 non-null   int64  
 3   page_sentence_count_raw    1208 non-null   int64  
 4   page_token_count           1208 non-null   float64
 5   text                       1208 non-null   object 
 6   sentences                  1208 non-null   object 
 7   page_sentence_count_spacy  1208 non-null   int64  
 8   sentence_chunks            1208 non-null   object 
 9   num_chunks                 1208 non-null   int64  
dtypes: float64(1), int64(6), object(3)
memory usage: 94.5+ KB


In [19]:
df.head()

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,text,sentences,page_sentence_count_spacy,sentence_chunks,num_chunks
0,-41,29,4,1,7.25,Human Nutrition: 2020 Edition,[Human Nutrition: 2020 Edition],1,[[Human Nutrition: 2020 Edition]],1
1,-40,0,1,1,0.00,,[],0,[],0
2,-39,320,54,1,80.00,Human Nutrition: 2020 Edition UNIVERSITY OF ...,[Human Nutrition: 2020 Edition UNIVERSITY OF...,1,[[Human Nutrition: 2020 Edition UNIVERSITY O...,1
3,-38,212,32,1,53.00,Human Nutrition: 2020 Edition by University of...,[Human Nutrition: 2020 Edition by University o...,1,[[Human Nutrition: 2020 Edition by University ...,1
4,-37,797,145,2,199.25,Contents Preface University of Hawai‘i at Mā...,[Contents Preface University of Hawai‘i at M...,2,[[Contents Preface University of Hawai‘i at ...,1


In [26]:
# splitting each chunk into its own item

import re

pages_and_chunks = []

for item in tqdm(pages_and_texts):
  for sentence_c in item['sentence_chunks']:
    chunk_dict = {}

    chunk_dict['page_number'] = item['page_number']

    joined_sent_chunk = "".join(sentence_c).replace("  ", " ").strip()
    joined_sent_chunk = re.sub(r'\.([A-Z])', r'. \1', joined_sent_chunk)

    chunk_dict['sentence_chunk'] = joined_sent_chunk

    chunk_dict['chunk_char_count'] = len(joined_sent_chunk)

    chunk_dict['chunk_word_count'] = len([word for word in joined_sent_chunk.split(" ")])
    chunk_dict['chunk_token_count'] = len(joined_sent_chunk) / 4

    pages_and_chunks.append(chunk_dict)

  0%|          | 0/1208 [00:00<?, ?it/s]

In [30]:
len(pages_and_chunks)

1843

In [31]:
random.sample(pages_and_chunks, 1)

[{'page_number': 816,
  'sentence_chunk': 'milk is the best source to fulfill nutritional requirements. An exclusively breastfed infant does not even need extra water, including in hot climates. A newborn infant (birth to 28 days) requires feedings eight to twelve times a day or more. Between 1 and 3 months of age, the breastfed infant becomes more efficient, and the number of feedings per day often become fewer even though the amount of milk consumed stays the same. After about six months, infants can gradually begin to consume solid foods to help meet nutrient needs. Foods that are added in addition to breastmilk are called complementary foods. Complementary foods should be nutrient dense to provide optimal nutrition. Complementary foods include baby meats, vegetables, fruits, infant cereal, and dairy products such as yogurt, but not infant formula. Infant formula is a substitute, not a complement to breastmilk. In addition to complementary foods, the World Health Organization recomm

In [32]:
df = pd.DataFrame(pages_and_chunks)

df.describe()

,page_number,chunk_char_count,chunk_word_count,chunk_token_count
count,1843.000000,1843.000000,1843.000000,1843.000000
mean,583.381443,734.829626,112.720022,183.707406
std,347.788670,447.431540,71.065283,111.857885
min,-41.000000,12.000000,3.000000,3.000000
25%,280.500000,315.000000,45.000000,78.750000
50%,586.000000,746.000000,114.000000,186.500000
75%,890.000000,1118.500000,173.000000,279.625000
max,1166.000000,1831.000000,297.000000,457.750000


In [38]:
# filter out chunks of text for short chunks

min_token_len = 30

for row in df[df['chunk_token_count'] <= min_token_len].sample(5).iterrows():
  print(f"Chunk token count : {row[1]['chunk_token_count']} | Text : {row[1]['sentence_chunk']}")

Chunk token count : 25.0 | Text : http:/ /www.ajcn.org/content/87/1/64.long. Accessed September 22, 2017. 554 | Water-Soluble Vitamins
Chunk token count : 27.5 | Text : Figure 15.1 reused “Silohuette of Three People” by photo- nic.co.uk nic / Unsplash License 1158 | Attributions
Chunk token count : 26.5 | Text : It is stored in the rectum until it is expelled through the anus via defecation. The Digestive System | 77
Chunk token count : 17.5 | Text : Published August 2011. Accessed September 22, 2017. Introduction | 147
Chunk token count : 24.5 | Text : http:/ /pressbooks.oer.hawaii.edu/ humannutrition2/?p=225 330 | Digestion and Absorption of Lipids


In [39]:
pages_and_chunks_over_min_token_len = df[df['chunk_token_count'] > min_token_len].to_dict(orient = "records")

pages_and_chunks_over_min_token_len[:2]



[{'page_number': -39,
  'sentence_chunk': 'Human Nutrition: 2020 Edition UNIVERSITY OF HAWAI‘I AT MĀNOA FOOD SCIENCE AND HUMAN NUTRITION PROGRAM ALAN TITCHENAL, SKYLAR HARA, NOEMI ARCEO CAACBAY, WILLIAM MEINKE-LAU, YA-YUN YANG, MARIE KAINOA FIALKOWSKI REVILLA, JENNIFER DRAPER, GEMADY LANGFELDER, CHERYL GIBBY, CHYNA NICOLE CHUN, AND ALLISON CALABRESE',
  'chunk_char_count': 308,
  'chunk_word_count': 42,
  'chunk_token_count': 77.0},
 {'page_number': -38,
  'sentence_chunk': 'Human Nutrition: 2020 Edition by University of Hawai‘i at Mānoa Food Science and Human Nutrition Program is licensed under a Creative Commons Attribution 4.0 International License, except where otherwise noted.',
  'chunk_char_count': 210,
  'chunk_word_count': 30,
  'chunk_token_count': 52.5}]

In [44]:
random.sample(pages_and_chunks_over_min_token_len, 1)

[{'page_number': 879,
  'sentence_chunk': 'schools, and also snacks to after-school facilities. School districts that take part receive subsidies from the US Department of Agriculture (USDA) for every meal they serve. School lunches must meet the 2015 Dietary Guidelines for Americans and need to provide one-third of the RDAs for protein, vitamin A, vitamin C, iron, and calcium. However, local authorities make the decisions about what foods to serve and how they are prepared.4 The Healthy School Lunch Campaign works to improve the food served to children in school and to promote children’s short- and long-term health by educating government officials, school officials, food-service workers, and parents. Sponsored by the Physicians Committee for Responsible Medicine, this organization encourages schools to offer more low-fat, cholesterol-free options in school cafeterias and in vending machines.5 Food Allergies and Food Intolerance As discussed previously, the development of food allergi

In [ ]:
## Embedding text chunks
# convert it into meaningful numerical representations

